In [274]:
from keras.layers import Input,Dense
from keras.models import Model,Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from keras import losses
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")
np.random.seed(203)

In [275]:
data = pd.read_csv("../data/train_data")
data.head()

,Class,Timestamp,IPVal,IPs,Domains,Hashes,Filesize
0,1,0,12398,8,10,12,52.18
1,2,1,66661,1,11,21,60.43
2,3,2,91231,1,5,10,102.11
3,1,3,29999,3,32,1,368.77
4,2,4,33333,12,56,1,11.88


In [276]:
X = data.drop(['Class'], axis=1).values
Y = data["Class"].values

In [373]:
input_layer = Input(shape=(X.shape[1],))

In [374]:
#encoded = Dense(10, activation='relu',activity_regularizer=regularizers.l1(10e-8))(input_layer)
#encoded = Dense(5, activation='relu')(encoded)
encoded = Dense(64, activation='relu',
                activity_regularizer=regularizers.l1(10e-9))(input_layer)

In [375]:
#decoded = Dense(5, activation='relu')(encoded)
#decoded = Dense(10, activation='relu')(decoded)
decoded = Dense(64, activation='relu')(encoded)

In [376]:
output_layer = Dense(X.shape[1], activation='relu')(decoded)
#output_layer = Dense(X.shape[1], activation='sigmoid')(encoded)

In [377]:
autoencoder = Model(input_layer,output_layer)
autoencoder.compile(optimizer="adadelta",loss=losses.binary_crossentropy)

In [378]:
x = data.drop(["Class"], axis=1)
y = data["Class"].values

x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_1, x_2, x_3 = x_scale[y == 1], x_scale[y == 2], x_scale[y == 3]

In [379]:
autoencoder.fit(x_1,x_1,
                batch_size=256, epochs=50,
                shuffle = True, validation_split = 0.20)

Train on 28 samples, validate on 7 samples
Epoch 1/50
28/28 [==============================] - 1s 48ms/step - loss: 3.6921 - val_loss: 3.3513
Epoch 2/50
28/28 [==============================] - 0s 214us/step - loss: 3.3513 - val_loss: 2.7494
Epoch 3/50
28/28 [==============================] - 0s 213us/step - loss: 2.7494 - val_loss: 2.7037
Epoch 4/50
28/28 [==============================] - 0s 214us/step - loss: 2.7037 - val_loss: 2.6851
Epoch 5/50
28/28 [==============================] - 0s 285us/step - loss: 2.6851 - val_loss: 2.6713
Epoch 6/50
28/28 [==============================] - 0s 214us/step - loss: 2.6713 - val_loss: 2.6600
Epoch 7/50
28/28 [==============================] - 0s 249us/step - loss: 2.6600 - val_loss: 2.6503
Epoch 8/50
28/28 [==============================] - 0s 249us/step - loss: 2.6503 - val_loss: 2.4463
Epoch 9/50
28/28 [==============================] - 0s 284us/step - loss: 2.4463 - val_loss: 2.0921
Epoch 10/50
28/28 [==============================] - 0s 28

In [380]:
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

In [381]:
one_hid_rep=hidden_representation.predict(x_1)
two_hid_rep=hidden_representation.predict(x_2)
three_hid_rep=hidden_representation.predict(x_3)

In [382]:
rep_x = np.append(one_hid_rep,two_hid_rep,axis=0)
rep_x = np.append(rep_x,three_hid_rep,axis=0)
y_1 = np.full(x_1.shape[0],1)
y_2 = np.full(x_2.shape[0],2)
y_3 = np.full(x_3.shape[0],3)
rep_y = np.append(y_1,y_2)
rep_y = np.append(rep_y,y_3)

In [383]:
rep_x

array([[0.        , 0.        , 0.05942309, ..., 0.        , 0.023955  ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00381565, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.06045492, ..., 0.        , 0.02586344,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00694304, 0.        ,
        0.        ]], dtype=float32)

In [384]:
train_x, val_x, train_y, val_y = train_test_split(rep_x,rep_y,test_size=0.25)

In [385]:
clf = LogisticRegression(multi_class='multinomial',solver='lbfgs').fit(train_x,train_y)
#clf = LinearRegression().fit(train_x,train_y)
pred_y = clf.predict(val_x)

print("\nClassification Report: ")
print(classification_report(val_y,pred_y))

print("\nAccuracy Score: ",accuracy_score(val_y,pred_y))


Classification Report: 
              precision    recall  f1-score   support

           1       0.14      0.60      0.22         5
           2       0.20      0.09      0.13        11
           3       0.00      0.00      0.00        11

    accuracy                           0.15        27
   macro avg       0.11      0.23      0.12        27
weighted avg       0.11      0.15      0.09        27


Accuracy Score:  0.14814814814814814


C:\Users\KingKwan\Anaconda2\envs\FirstStep\venv2\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [386]:
val_y

array([2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 3, 1, 1, 3, 1, 2, 1, 3, 3, 2, 2,
       1, 3, 3, 2, 3])

In [387]:
pred_y

array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 1, 1, 1])

In [ ]:
new = pd.read_csv('../data/valid_data')
new

In [ ]:
res = autoencoder.predict(new)

In [ ]:
res